In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from sklearn.model_selection import train_test_split
import os
import openai
from openai import OpenAIError
from openai import OpenAI
import pandas as pd
import time
import json
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import krippendorff
import torch
from transformers import TrainingArguments, Trainer, EvalPrediction, AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, IntervalStrategy
from datasets import Dataset, DatasetDict
from datetime import datetime

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/ulugsali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ulugsali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# loading data
df = pd.read_csv('dataset_tiktok-comments-scraper_2024-05-03_19-52-46-840.csv')

# we only need a text column in tiktok_comments dataset since it's the only
# thing in this dataset that can help us gauge the brand perception. 
df = df[['text']]

df.head()

,text
0,Why she look like LUCKI with the fit on
1,Jenn carter biggest opp
2,“What’s jack” LMAOO w moms
3,Yoo she was gettin jiggy wit
4,Nah she valid 😂


In [9]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Preprocess and clean data
df['cleaned_text'] = df['text'].apply(clean_text)
df['filtered_text'] = df['cleaned_text'].apply(remove_stopwords)
df['language'] = df['filtered_text'].apply(detect_language)
df = df[df['language'] == 'en']  # Keep only English language comments

# Handle missing values
df.dropna(subset=['filtered_text'], inplace=True)

df.head()

,text,cleaned_text,filtered_text,language
8,that’s lucki 😭,thats lucki,thats lucki,en
10,Lucky kid!! You’re a great stepdad Stretch!!,lucky kid youre a great stepdad stretch,lucky kid youre great stepdad stretch,en
12,Y’all cray I’m not spending 800 on shoes😳😳 the...,yall cray im not spending 800 on shoes they ni...,yall cray im spending 800 shoes nice tho lol,en
13,It’s Mr. Stretch cough that did it for me 🤣💀💀,its mr stretch cough that did it for me,mr stretch cough,en
14,I been trying to order your mum's knife set bu...,i been trying to order your mums knife set but...,trying order mums knife set doesnt ship australia,en


In [10]:
df = df[['filtered_text']]

# Optionally, if you want to rename the column to something more general like 'text', you can do:
df.rename(columns={'filtered_text': 'text'}, inplace=True)

# Now 'df' contains only the column with the cleaned and filtered text
df.head()

,text
8,thats lucki
10,lucky kid youre great stepdad stretch
12,yall cray im spending 800 shoes nice tho lol
13,mr stretch cough
14,trying order mums knife set doesnt ship australia


In [12]:
df.to_csv('filtered_amiri_data.csv', index=False)